In [36]:
#!pip install osmnx folium geopy pandas --upgrade

In [37]:
# Importar las librerías necesarias
import osmnx as ox
import networkx as nx
import folium
import pandas as pd
import heapq
from geopy.distance import geodesic

In [38]:
def clean_graph(G, network_type):
    max_speed = {'drive': 40, 'walk': 5}
    for u, v, k in G.edges(keys=True):
        length = G.edges[u, v, k]["length"]
        max_speed_value = max_speed[network_type]

        # Penalización por tipo de calle
        penalty = 1.5 if "highway" in G.edges[u, v, k] and G.edges[u, v, k]["highway"] in ["residential", "tertiary"] else 1.0

        G.edges[u, v, k]["maxspeed"] = max_speed_value
        G.edges[u, v, k]["weight"] = (length / max_speed_value) * penalty
    return G

In [39]:
# Función para inicializar estilos de aristas
def initialize_edge_styles(G):
    for edge in G.edges:
        G.edges[edge]["color"] = "#d36206"
        G.edges[edge]["alpha"] = 0.2
        G.edges[edge]["linewidth"] = 0.5

In [40]:
# Solicitar las coordenadas de inicio y destino
def get_coordinates():
    while True:
        try:
            coords = input("Ingrese las coordenadas (latitud, longitud): ").strip()
            lat, lon = map(float, coords.split(","))
            return (lat, lon)
        except ValueError:
            print("Entrada inválida. Por favor, ingrese las coordenadas en el formato 'lat,lon'.")

In [41]:
# Solicitar el tipo de transporte
def get_transport_type():
    while True:
        print("Ingrese el tipo de viaje que realizará:")
        print("  1. Auto")
        print("  2. Caminando")
        try:
            option = int(input("Seleccione una opción (1 o 2): ").strip())
            if option in [1, 2]:
                return 'drive' if option == 1 else 'walk'
            else:
                print("Opción inválida. Por favor seleccione 1 o 2.")
        except ValueError:
            print("Entrada inválida. Por favor ingrese un número (1 o 2).")

In [42]:
# Algoritmo de Dijkstra
def dijkstra(G, orig, dest):
    for node in G.nodes:
        G.nodes[node]["visited"] = False
        G.nodes[node]["distance"] = float("inf")
        G.nodes[node]["previous"] = None
    G.nodes[orig]["distance"] = 0
    pq = [(0, orig)]
    while pq:
        _, node = heapq.heappop(pq)
        if node == dest:
            break
        if G.nodes[node]["visited"]:
            continue
        G.nodes[node]["visited"] = True
        for u, v, k in G.out_edges(node, keys=True):
            weight = G.edges[u, v, k]["weight"]
            if G.nodes[v]["distance"] > G.nodes[node]["distance"] + weight:
                G.nodes[v]["distance"] = G.nodes[node]["distance"] + weight
                G.nodes[v]["previous"] = node
                heapq.heappush(pq, (G.nodes[v]["distance"], v))

In [43]:
# Algoritmo A*
def a_star(G, orig, dest):
    def heuristic(node1, node2):
        x1, y1 = G.nodes[node1]["x"], G.nodes[node1]["y"]
        x2, y2 = G.nodes[node2]["x"], G.nodes[node2]["y"]
        return geodesic((y1, x1), (y2, x2)).meters  # Distancia geodésica en metros

    for node in G.nodes:
        G.nodes[node]["g_score"] = float("inf")
        G.nodes[node]["f_score"] = float("inf")
        G.nodes[node]["previous"] = None

    G.nodes[orig]["g_score"] = 0
    G.nodes[orig]["f_score"] = heuristic(orig, dest)
    pq = [(G.nodes[orig]["f_score"], orig)]

    while pq:
        _, node = heapq.heappop(pq)
        if node == dest:
            break
        for u, v, k in G.out_edges(node, keys=True):
            tentative_g_score = G.nodes[node]["g_score"] + G.edges[u, v, k]["weight"]
            if tentative_g_score < G.nodes[v]["g_score"]:
                G.nodes[v]["g_score"] = tentative_g_score
                G.nodes[v]["f_score"] = tentative_g_score + heuristic(v, dest)
                G.nodes[v]["previous"] = node
                heapq.heappush(pq, (G.nodes[v]["f_score"], v))


In [44]:
# Mostrar resultados en tabla
def show_results_in_table(total_time, total_distance):
    """
    Muestra los resultados finales en una tabla.
    """
    data = {
        "Algoritmo": ["Dijkstra", "A*"],
        "Distancia Total (km)": [
            total_distance["Dijkstra"] / 1000,  # Convertir de metros a kilómetros
            total_distance["A*"] / 1000
        ],
        "Tiempo Total (minutos)": [
            total_time["Dijkstra"],
            total_time["A*"]
        ]
    }

    # Crear el DataFrame y mostrarlo
    df_results = pd.DataFrame(data)
    display(df_results)


In [45]:
# Obtener las coordenadas de la ruta más corta
def get_route_coordinates(G, orig, dest):
    path = []
    curr = dest
    while curr != orig:
        prev = G.nodes[curr]["previous"]
        if prev is None:
            print("No se pudo encontrar un camino desde el origen al destino.")
            return None
        path.append((G.nodes[curr]["y"], G.nodes[curr]["x"]))
        curr = prev
    path.append((G.nodes[orig]["y"], G.nodes[orig]["x"]))
    path.reverse()
    return path

In [46]:
# Visualizar la ruta con Folium
def plot_route_with_folium(G, orig, dest, route_coordinates):
    folium_map = folium.Map(
        location=[G.nodes[orig]['y'], G.nodes[orig]['x']],
        zoom_start=14,
        tiles='OpenStreetMap'
    )
    folium.Marker(
        location=[G.nodes[orig]['y'], G.nodes[orig]['x']],
        popup="Inicio",
        icon=folium.Icon(color="green", icon="play"),
    ).add_to(folium_map)
    folium.Marker(
        location=[G.nodes[dest]['y'], G.nodes[dest]['x']],
        popup="Destino",
        icon=folium.Icon(color="red", icon="stop"),
    ).add_to(folium_map)
    folium.PolyLine(
        route_coordinates,
        color="blue",
        weight=5,
        opacity=0.8,
        tooltip="Ruta más corta"
    ).add_to(folium_map)
    return folium_map

In [47]:
# Coordenadas guardadas
saved_coordinates = {
    "UPIIT": (19.323118, -98.233548),
    "Parque de Zacatelco": (19.215691, -98.240524),
    "Soriana Ocotlan": (19.318605, -98.220713),
    "Zoologico del Altiplano": (19.338439, -98.199046),
}

# Función para obtener coordenadas (seleccionar de lista o ingresar manualmente)
def get_coordinates_with_saved_options():
    print("\n¿Desea seleccionar un punto de la lista predefinida o ingresar coordenadas manualmente?")
    print("  1. Seleccionar de la lista")
    print("  2. Ingresar manualmente")
    while True:
        try:
            option = int(input("Seleccione una opción (1 o 2): ").strip())
            if option == 1:
                print("\nPuntos disponibles:")
                for idx, (name, coord) in enumerate(saved_coordinates.items(), start=1):
                    print(f"  {idx}. {name} - Coordenadas: {coord}")
                while True:
                    try:
                        choice = int(input("Seleccione un punto por su número: ").strip())
                        if 1 <= choice <= len(saved_coordinates):
                            selected_name = list(saved_coordinates.keys())[choice - 1]
                            print(f"Ha seleccionado: {selected_name} - Coordenadas: {saved_coordinates[selected_name]}")
                            return saved_coordinates[selected_name]
                        else:
                            print("Por favor, seleccione un número válido de la lista.")
                    except ValueError:
                        print("Entrada inválida. Por favor, ingrese un número.")
            elif option == 2:
                return get_coordinates()  # Llama a la función manual existente
            else:
                print("Por favor, seleccione una opción válida (1 o 2).")
        except ValueError:
            print("Entrada inválida. Por favor, ingrese un número.")


In [48]:
# Función para agregar puntos dinámicamente
def get_multiple_points():
    points = {}  # Diccionario para almacenar los puntos con sus nombres
    point_counter = 0  # Contador para nombrar los puntos dinámicamente (A, B, C, etc.)

    while True:
        # Generar el nombre del punto (A, B, C, ...)
        point_name = chr(65 + point_counter)  # 65 es el código ASCII de 'A'
        print(f"\nIngrese las coordenadas para el punto {point_name}:")

        # Obtener coordenadas
        coord = get_coordinates_with_saved_options()
        points[point_name] = coord  # Guardar el punto en el diccionario

        # Preguntar si desea agregar otro punto
        while True:
            add_more = input(f"¿Desea agregar otro punto después de {point_name}? (s/n): ").strip().lower()
            if add_more in ["s", "n"]:
                break
            print("Por favor, ingrese 's' para sí o 'n' para no.")

        if add_more == "n":
            break  # Finalizar el bucle si el usuario no desea agregar más puntos

        point_counter += 1  # Incrementar el contador para el próximo punto

    return points


In [49]:
import random

def generate_random_color():
    """
    Genera un color hexadecimal aleatorio.
    """
    return f"#{random.randint(0, 255):02x}{random.randint(0, 255):02x}{random.randint(0, 255):02x}"

def plot_full_route_with_folium(G, nodes, routes, algorithm_name):
    """
    Genera un mapa para un algoritmo específico con puntos y rutas.
    """
    # Crear el mapa centrado en el primer punto
    first_point = list(nodes.values())[0]
    folium_map = folium.Map(
        location=[G.nodes[first_point]['y'], G.nodes[first_point]['x']],
        zoom_start=12,
        tiles='OpenStreetMap'
    )

    # Colores predefinidos para puntos
    predefined_colors = ["blue", "red", "green", "purple", "orange", "pink"]
    color_map = {}

    # Añadir puntos al mapa
    legend_items = []  # Para la leyenda
    for idx, (name, node) in enumerate(nodes.items()):
        if idx < len(predefined_colors):
            color = predefined_colors[idx]
        else:
            # Generar un color aleatorio si se exceden los colores predefinidos
            color = generate_random_color()
        color_map[name] = color  # Guardar el color asignado para la leyenda

        # Añadir el marcador del punto
        folium.Marker(
            location=[G.nodes[node]['y'], G.nodes[node]['x']],
            popup=f"Punto {name}",
            icon=folium.Icon(color=color if idx < len(predefined_colors) else "lightgray", icon="circle"),
        ).add_to(folium_map)

        # Añadir el punto a la leyenda
        legend_items.append(f"<span style='color:{color};'>Punto {name}</span>")

    # Añadir rutas al mapa
    for start_name, end_name, route_coordinates in routes:
        folium.PolyLine(
            route_coordinates,
            color="blue",  # Color de las rutas para el algoritmo
            weight=5,
            opacity=0.8,
            tooltip=f"Ruta de {start_name} a {end_name} ({algorithm_name})"
        ).add_to(folium_map)

    # Añadir leyenda al mapa
    legend_html = """
    <div style="position: fixed; bottom: 50px; left: 50px; z-index: 1000; background-color: white; padding: 10px; border: 2px solid black;">
    <h4>Leyenda</h4>
    """
    legend_html += "".join(f"<p style='margin:0'>{item}</p>" for item in legend_items)
    legend_html += "</div>"
    folium_map.get_root().html.add_child(folium.Element(legend_html))

    return folium_map



In [50]:
# Programa principal con impresión de tiempos y distancias
try:
    print("Seleccione los puntos de la ruta:")
    points = get_multiple_points()  # Obtener todos los puntos ingresados por el usuario
    print("\nPuntos ingresados:")
    for name, coord in points.items():
        print(f"  {name}: {coord}")

    # Tipo de transporte
    network_type = get_transport_type()

    print("\nDescargando el grafo de Tlaxcala...")
    tlaxcala_graph = ox.graph_from_place("Tlaxcala, México", network_type=network_type)
    G = clean_graph(tlaxcala_graph, network_type)
    initialize_edge_styles(G)

    # Obtener los nodos más cercanos para cada punto
    nodes = {}
    for name, coord in points.items():
        nodes[name] = ox.distance.nearest_nodes(G, coord[1], coord[0])
        print(f"Nodo más cercano al punto {name}: {nodes[name]}")

    # Rutas calculadas por algoritmo
    routes_by_algorithm = {"Dijkstra": [], "A*": []}
    point_names = list(points.keys())  # Lista de nombres de los puntos (A, B, C, ...)

    # Variables para tiempo y distancia totales
    total_time = {"Dijkstra": 0, "A*": 0}
    total_distance = {"Dijkstra": 0, "A*": 0}

    for i in range(len(point_names) - 1):
        start_name = point_names[i]
        end_name = point_names[i + 1]
        print(f"\nCalculando la ruta de {start_name} a {end_name}...")

        start_node = nodes[start_name]
        end_node = nodes[end_name]

        # Calcular rutas para cada algoritmo
        algorithms = {"Dijkstra": dijkstra, "A*": a_star}
        for algorithm_name, algorithm in algorithms.items():
            print(f"Ejecutando el algoritmo {algorithm_name} de {start_name} a {end_name}...")
            try:
                # Ejecutar el algoritmo
                algorithm(G, start_node, end_node)
                path = []
                curr = end_node
                while curr != start_node:
                    prev = G.nodes[curr]["previous"]
                    path.append((prev, curr))
                    curr = prev
                path.reverse()

                # Calcular distancia total y tiempo
                try:
                    dist = sum(G.edges[u, v, 0]["length"] for u, v in path)
                    avg_speed = 40 if network_type == 'drive' else 5
                    time = (dist / 1000) / avg_speed * 60

                    # Guardar en totales
                    total_distance[algorithm_name] += dist
                    total_time[algorithm_name] += time

                    # Imprimir resultados entre puntos
                    print(f"{algorithm_name}:")
                    print(f"  Distancia estimada de {start_name} a {end_name}: {dist / 1000:.2f} km")
                    print(f"  Tiempo estimado de {start_name} a {end_name}: {time:.2f} minutos")

                    # Obtener coordenadas para el mapa
                    route_coordinates = get_route_coordinates(G, start_node, end_node)
                    routes_by_algorithm[algorithm_name].append((start_name, end_name, route_coordinates))

                except Exception as e:
                    print(f"Error al calcular la distancia o tiempo para {algorithm_name}: {e}")

            except Exception as e:
                print(f"Error durante la ejecución de {algorithm_name}: {e}")

    # Imprimir totales finales
    print("\n--- Totales Finales ---")
    for algorithm_name in algorithms.keys():
        print(f"{algorithm_name}:")
        print(f"  Distancia total estimada: {total_distance[algorithm_name] / 1000:.2f} km")
        print(f"  Tiempo total estimado: {total_time[algorithm_name]:.2f} minutos")

    show_results_in_table(total_time, total_distance)

    # Mostrar mapas por separado
    for algorithm_name, routes in routes_by_algorithm.items():
        print(f"\nMostrando mapa para el algoritmo {algorithm_name}...")
        folium_map = plot_full_route_with_folium(G, nodes, routes, algorithm_name)
        display(folium_map)

except Exception as e:
    print(f"Error general: {e}")


Seleccione los puntos de la ruta:

Ingrese las coordenadas para el punto A:

¿Desea seleccionar un punto de la lista predefinida o ingresar coordenadas manualmente?
  1. Seleccionar de la lista
  2. Ingresar manualmente


Seleccione una opción (1 o 2):  1



Puntos disponibles:
  1. UPIIT - Coordenadas: (19.323118, -98.233548)
  2. Parque de Zacatelco - Coordenadas: (19.215691, -98.240524)
  3. Soriana Ocotlan - Coordenadas: (19.318605, -98.220713)
  4. Zoologico del Altiplano - Coordenadas: (19.338439, -98.199046)


Seleccione un punto por su número:  1


Ha seleccionado: UPIIT - Coordenadas: (19.323118, -98.233548)


...

Seleccione una opción (1 o 2):  1



Descargando el grafo de Tlaxcala...
Nodo más cercano al punto A: 360340293
Nodo más cercano al punto B: 352143711

Calculando la ruta de A a B...
Ejecutando el algoritmo Dijkstra de A a B...
Dijkstra:
  Distancia estimada de A a B: 2.98 km
  Tiempo estimado de A a B: 4.47 minutos
Ejecutando el algoritmo A* de A a B...
A*:
  Distancia estimada de A a B: 3.70 km
  Tiempo estimado de A a B: 5.54 minutos

--- Totales Finales ---
Dijkstra:
  Distancia total estimada: 2.98 km
  Tiempo total estimado: 4.47 minutos
A*:
  Distancia total estimada: 3.70 km
  Tiempo total estimado: 5.54 minutos


,Algoritmo,Distancia Total (km),Tiempo Total (minutos)
0,Dijkstra,2.982399,4.473599
1,A*,3.696476,5.544715



Mostrando mapa para el algoritmo Dijkstra...



Mostrando mapa para el algoritmo A*...
